In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from zipfile import ZipFile
import matplotlib as mlp
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:

file_name = "/kaggle/input/facial-keypoints-detection/training.zip"
with ZipFile(file_name, 'r') as z:
  z.extractall(path="/kaggle/working/")
  print('done')

In [ ]:
file_name = "/kaggle/input/facial-keypoints-detection/test.zip"
with ZipFile(file_name, 'r') as z:
  z.extractall(path="/kaggle/working/")
  print('done')

In [ ]:
df_train = pd.read_csv("../input/facial-keypoints-detection/training.zip")
df_test = pd.read_csv("/kaggle/working/training.csv")

In [ ]:
df_train.info() # a lot of missing data - seems like only eye_center, nose_tip and bottom_lip will be accurate 

In [ ]:
feature_col, target_cols = 'Image', list(df_train.drop('Image', axis=1).columns)

In [ ]:
df_train[target_cols] = df_train[target_cols].fillna(df_train[target_cols].mean()) # any other/better way to do this?
df_train.info()

In [ ]:
IMG_WIDTH  = 96
IMG_HEIGHT = 96
IMG_CHANNELS = 1

# Split the `Image` column around delimiter `space` and 
# create a numpy array with `dtype=int`, finally reshape
# it according the defined height width, and channels
images = np.array(df_train[feature_col].str.split().tolist(), dtype='float').reshape(-1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
labels = df_train[target_cols].to_numpy()

In [ ]:
normalized_images = images / 255.

In [ ]:

def show_examples(images, landmarks):
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(16, 16))
    
    for img, marks, ax in zip(images, landmarks, axes.ravel()):
        # Keypoints
        x_points = marks[:: 2]
        y_points = marks[1::2]
        
        ax.imshow(img.squeeze(), cmap='gray')
        ax.scatter(x_points, y_points, s=10, color='red')
    
    plt.show()
    

idx = np.random.choice(16, 16)
show_examples(images[idx], labels[idx])

In [ ]:
from sklearn.model_selection import train_test_split
train_images, valid_images, train_labels, valid_labels = train_test_split(normalized_images, labels, test_size=0.1, random_state=7)

In [ ]:
from tensorflow.keras.layers import (
    Input,
    Conv2D, 
    MaxPool2D, 
    Dense, 
    BatchNormalization, 
    ReLU, 
    Dropout, 
    Flatten,
    Dropout,
    Concatenate,
    GlobalAvgPool2D
)
from tensorflow.keras.regularizers import L2


In [ ]:
def build_model():
    inputs = Input((96, 96, 1))
    
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D()(x)
    
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D()(x)
    
    x = Conv2D(filters=256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D()(x)
    
    x = Flatten()(x)
    x = Dense(2048,kernel_regularizer=L2(l2=0.05))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Dense(512, kernel_regularizer=L2(l2=0.02))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Dense(128, kernel_regularizer=L2(l2=0.01))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Dense(30)(x)
    
    model = tf.keras.Model(inputs, outputs=x)
    return model

In [ ]:
model = build_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

In [ ]:
ckp_filepath = 'trained-models/model'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=ckp_filepath, 
                                                      monitor='val_mae', 
                                                      mode='auto',
                                                      save_best_only=True, 
                                                      save_weights_only=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.9, monitor='val_mae', 
                                                 mode='auto', cooldown=0, patience=5, verbose=1, min_lr=1e-5)

In [ ]:
EPOCHS = 200
BATCH_SIZE = 256

# if Path(ckp_filepath).exists():
#     model.load_weights(ckp_filepath)
    
history = model.fit(train_images, 
                    train_labels, 
                    validation_data=(valid_images, valid_labels), 
                    batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[model_checkpoint, reduce_lr])

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(12, 12))

logs = history.history

n_epochs = range(len(logs['loss']))
train_loss, valid_loss = logs['loss'], logs['val_loss']
train_mae, valid_mae = logs['mae'], logs['val_mae']


ax1.plot(n_epochs, train_loss, label='train loss', linestyle='solid')
ax1.plot(n_epochs, valid_loss, label='validation loss', linestyle='dashed')
ax1.set(xlabel='Number of iterations', ylabel='Mean squared error', title='Training and Validation loss vs Number of epochs', yscale='log')
_ = ax1.legend()

ax2.plot(n_epochs, train_mae, label='train MAE', linestyle='solid')
ax2.plot(n_epochs, valid_mae, label='validation MAE', linestyle='dashed')
ax2.set(xlabel='Number of iterations', ylabel='Mean absolute error', title='Training and Validation MAE vs Number of epochs', yscale='log')
_ = ax2.legend()

In [ ]:
df_test.head(1)

In [ ]:
test_images = np.array(df_test['Image'].str.split().tolist(), dtype='int').reshape(-1, 96, 96, 1)

In [ ]:
normalized_test_images = test_images / 255.

In [ ]:
model.load_weights(ckp_filepath) # load the best weights
keypoints_predictions = model.predict(normalized_test_images, batch_size=BATCH_SIZE)

In [ ]:
idx = np.random.choice(16, 16)
show_examples(test_images[idx], keypoints_predictions[idx])